# Projet Notations de films 

Introduction, expliquer le contexte, démarche, objectif, etc ..................................

### importation des librairies et fichiers

In [1]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install lxml
!pip install aiohttp
!pip install nest_asyncio

import asyncio
import aiohttp
import pandas as pd
from bs4 import BeautifulSoup
import re
import nest_asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.6 MB/s eta 0:00:00
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 67.5 MB/s eta 0:00:00


In [2]:
# Importation des librairies

# import seaborn as sns #pas besoin si utilise fonctions
import matplotlib.pyplot as plt

# Regression
import statsmodels.api as sm
from statsmodels.formula.api import rlm 

In [3]:
import os

# Importation des fichiers python (contenant des fonctions) crées dans le cadre des 3 parties du projet :

for f in os.listdir('A_Base_de_données'):
    if f.endswith('.py'):
        module_name = os.path.splitext(f)[0]
        exec(f"from A_Base_de_données.{module_name} import *")


for f in os.listdir('B_Analyse_descriptive'):
    if f.endswith('.py'):
        module_name = os.path.splitext(f)[0]
        exec(f"from B_Analyse_descriptive.{module_name} import *")


for f in os.listdir('C_Modele'):
    if f.endswith('.py'):
        module_name = os.path.splitext(f)[0]
        exec(f"from C_Modele.{module_name} import *")


## I - Création de la base de données

### Webscrapping AlloCiné

In [4]:
# Récupération d'une liste de films qui possèdent tous une note moyenne de spectateur et une note moyenn de presse entre l'année .... et l'année .....
base_films = get_base_films(2010, 2010)


# Récupération pour chaque film de différentes caractériques
nest_asyncio.apply()
base_films_enrichie, presse = get_carac_film(base_films)


nombre de films récupérés :  485


In [5]:
base_films_enrichie

,titre,identifiant,date,durée,auteur,spectateur,presse,genre1,genre2,genre3,...,date_sortie_bluray,date_sortie_vod,type_film,budget,langues,format_production,couleur,format_audio,format_projection,num_visa
0,Rien à déclarer,173528,26 janvier 2011,1h 48min,Dany Boon,"2,4","2,2",Comédie,,,...,15/06/2011,15/06/2011,Long métrage,-,"Flamand, Français",-,Couleur,-,-,125668
1,Rien à déclarer,173528,26 janvier 2011,1h 48min,Dany Boon,"2,4","2,2",Comédie,,,...,None,None,None,None,None,None,None,None,None,None
2,Les petits mouchoirs,146632,20 octobre 2010,2h 34min,Guillaume Canet,"3,6","2,9",Comédie dramatique,,,...,23/02/2011,01/03/2016,Long métrage,17 000 000 EUR,Français,-,Couleur,-,-,121 898
3,Les petits mouchoirs,146632,20 octobre 2010,2h 34min,Guillaume Canet,"3,6","2,9",Comédie dramatique,,,...,None,None,None,None,None,None,None,None,None,None
4,L'Agence tous risques,133588,16 juin 2010,1h 54min,Joe Carnahan,"3,2","3,2",Action,Comédie,,...,03/11/2010,01/06/2015,Long métrage,110 000 000 USD,"Anglais, Français, Espagnol, Swahili",-,Couleur,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,Ich bin eine Terroristin,198778,6 juin 2012,1h 37min,Valerie Gaudissart,"2,7","2,5",Comédie,Famille,,...,None,None,None,None,None,None,None,None,None,None
932,Exit una storia personale,186074,23 février 2011,1h 20min,Massimiliano Amato,"3,0","2,8",Drame,,,...,-,27/09/2018,Long métrage,-,"Anglais, Italien",HD,Couleur,Dolby SR,-,-
933,Exit una storia personale,186074,23 février 2011,1h 20min,Massimiliano Amato,"3,0","2,8",Drame,,,...,None,None,None,None,None,None,None,None,None,None
934,"Ab Irato, sous l'empire de la colère",201655,6 février 2013,1h 42min,Dominique Boccarossa,"2,5","2,7",Drame,,,...,-,-,Long métrage,-,Français,-,Couleur,-,-,125903


In [ ]:
# Exportation en CSV => mettre en commentaire lors du rendu afin que cela soit toujours le même CSV qui soit utilisé par la suite




### Ajout, Traitement et création de variables

In [ ]:
# Lecture des CSV : 

# Base de films : contenant diverses caractéristiques sur les films



# Base presse : contenant la note de chaque presse pour chaque film



Base presse

In [11]:
presse_notes = equivalence_notes(presse)
presse_notes

,CinemaTeaser,Télé 7 Jours,Excessif,La Croix,Les Fiches du Cinéma,L'Obs,Studio Ciné Live,Les Inrockuptibles,Libération,Elle,...,La Voix du Nord,France Soir,Critikat.com,Le Figaroscope,Le Point,20 Minutes,L'Humanité,Filmsactu,Marianne,Première
173528,,4,3,1,,,2,1,1,,...,,,1,1,3,3,,3,,3
146632,,5,3,3,,2,5,1,1,4,...,,,2,5,4,4,,3,,4
133588,,4,3,,,,,3,2,,...,,,3,4,3,4,,4,,
132039,,4,5,3,,3,5,4,3,5,...,,,,3,5,4,3,4,5,5
179349,,5,,4,,,2,4,4,,...,,,3,4,5,4,,,,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134462,,,,,,,,,,,...,,,,,,,,,,
130423,,,,,,,,,,,...,,,,,,,,,,
198778,,,,,,,,,,,...,,,,,,,,,,
186074,,,1,,,,,,4,,...,,,3,4,,,5,,,3


base films

In [6]:
# Ajout du genre des réalisateurs
table_avec_genre = get_genre_individuel(base_films, 'auteur')

# Aperçu du nb de réalisateurs m vs f
table_avec_genre['genre_ind'].value_counts()


genre_ind
m      323
f       36
m,f     15
f,m      3
Name: count, dtype: int64

In [7]:
table_avec_genre.head()

,genre_ind,titre,identifiant,date,durée,auteur,spectateur,presse,genre1,genre2,genre3
0,m,Very Bad Cops,144195,27 octobre 2010,1h 47min,Adam McKay,"2,5","3,3",Action,Comédie,
1,f,Adrienn Pàl,130423,25 juillet 2012,2h 16min,Agnes Kocsis,"2,6","3,6",Drame,,
2,m,Une vie de chat,125119,15 décembre 2010,1h 10min,Alain Gagnol,"3,8","3,6",Animation,Policier,
3,m,Le Plan B,141699,19 mai 2010,1h 47min,Alan Poul,"2,4","2,2",Comédie,Romance,
4,m,Le Livre d'Eli,128955,20 janvier 2010,1h 49min,Albert Hughes,"3,3","2,4",Science Fiction,,


In [8]:
# Transformation des types de films en variables dummy 
dummies1 = pd.get_dummies(table_avec_genre['genre1'], prefix='genre')
dummies2 = pd.get_dummies(table_avec_genre['genre2'], prefix='genre')
dummies3 = pd.get_dummies(table_avec_genre['genre3'], prefix='genre')

dummies1 = dummies1.astype(int)
dummies2 = dummies2.astype(int)
dummies3 = dummies3.astype(int)

dummies_sum = dummies1.add(dummies2, fill_value=0).add(dummies3, fill_value=0)
dummies_sum = dummies_sum.drop(columns=['genre_'])

df = pd.concat([table_avec_genre, dummies_sum], axis=1)
print('nombre de films pris en compte :', len(df))

nombre de films pris en compte : 377


In [9]:
# Conversion des notes en décimale
table_avec_genre['spectateur'] = table_avec_genre['spectateur'].apply(mise_en_forme_decimale)
table_avec_genre['presse'] = table_avec_genre['presse'].apply(mise_en_forme_decimale)

In [10]:
# Création variable : différence de notation entre spectateur et presse
table_avec_genre['diff_notation'] = table_avec_genre['spectateur']-table_avec_genre['presse']
table_avec_genre.head()


,genre_ind,titre,identifiant,date,durée,auteur,spectateur,presse,genre1,genre2,genre3,diff_notation
0,m,Very Bad Cops,144195,27 octobre 2010,1h 47min,Adam McKay,2.5,3.3,Action,Comédie,,-0.8
1,f,Adrienn Pàl,130423,25 juillet 2012,2h 16min,Agnes Kocsis,2.6,3.6,Drame,,,-1.0
2,m,Une vie de chat,125119,15 décembre 2010,1h 10min,Alain Gagnol,3.8,3.6,Animation,Policier,,0.2
3,m,Le Plan B,141699,19 mai 2010,1h 47min,Alan Poul,2.4,2.2,Comédie,Romance,,0.2
4,m,Le Livre d'Eli,128955,20 janvier 2010,1h 49min,Albert Hughes,3.3,2.4,Science Fiction,,,0.9


In [11]:
table_avec_genre = get_annee(table_avec_genre, 'date')

In [12]:
# Conversion de la durée en minutes
table_avec_genre['durée'] = table_avec_genre['durée'].astype('string')
table_avec_genre['duree_min'] = table_avec_genre['durée'].apply(duree_en_minutes)

Base CNC

## II -  Analyse descriptive

Axes :

- Analyses sur tous les films

- Analyses sur les films français

### Généralités

## Test de modèles sur la différence de notation

### Regression

- Notes spectateurs

In [122]:
# Définir les variables dépendantes et indépendantes
X = df[['duree_min', 'genre_Action', 'genre_Animation', 'genre_Policier', 'genre_Thriller', 'genre_Science Fiction', 'genre_Drame', 'genre_Aventure']]
y = df['spectateur']
X = sm.add_constant(X)

# Régression linéaire OLS
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             spectateur   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     15.89
Date:                Sun, 03 Nov 2024   Prob (F-statistic):           2.22e-19
Time:                        19:22:41   Log-Likelihood:                -256.63
No. Observations:                 296   AIC:                             531.3
Df Residuals:                     287   BIC:                             564.5
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.71

- Notes presse

In [124]:
# Définir les variables dépendantes et indépendantes
X = df[['duree_min', 'genre_Action', 'genre_Animation', 'genre_Policier', 'genre_Thriller', 'genre_Science Fiction', 'genre_Drame', 'genre_Aventure']]
y = df['presse']
X = sm.add_constant(X)

# Régression linéaire OLS
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 presse   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     17.38
Date:                Sun, 03 Nov 2024   Prob (F-statistic):           4.49e-21
Time:                        19:23:34   Log-Likelihood:                -216.55
No. Observations:                 296   AIC:                             451.1
Df Residuals:                     287   BIC:                             484.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.29

- Différences de notations

In [ ]:
# Définir les variables dépendantes et indépendantes
X = df[['duree_min', 'genre_Action', 'genre_Animation', 'genre_Policier', 'genre_Thriller', 'genre_Science Fiction', 'genre_Drame', 'genre_Aventure']]
y = df['diff_notation']
X = sm.add_constant(X)

# Régression linéaire OLS
model = sm.OLS(y, X).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:          diff_notation   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     1.929
Date:                Sun, 03 Nov 2024   Prob (F-statistic):             0.0556
Time:                        19:25:00   Log-Likelihood:                -226.63
No. Observations:                 296   AIC:                             471.3
Df Residuals:                     287   BIC:                             504.5
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.57